# 機械学習の手順
1. import
2. データを訓練データとテストデータに分ける
3. モデルを作って学習する
4. 判定

## 1. import

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.svm import SVC
from sklearn import metrics

## 2. データを訓練データとテストデータに分ける

In [2]:
# cat: 0, dog: 1
train_data = pd.read_csv("dc_photos/train/train_data.csv")
test_data = pd.read_csv("dc_photos/test/test_data.csv")

#### 説明変数（X）に画像データ、目的変数（y）にCSVファイルのDC列のデータを読み込む

```
前処理: 画像はモノクロにする。
    
水増し: 左右反転したもの,上下反転したもの,180度回転させたものを作り4倍に水増し。
```

- 訓練データの作成

In [3]:
train_len = len(train_data)

# 後でデータを入れる空の配列を作成(画像は75ピクセル)
X_train = np.empty((train_len * 4, 75 * 75), dtype=np.uint8)
y_train = np.empty(train_len * 4, dtype=np.uint8)

for i in range(len(train_data)):
    name = train_data.loc[i, "File name"]
    train_img = Image.open(f"dc_photos/train/{name}.jpg").convert("L")
    train_img = np.array(train_img)
    train_img_f = train_img.flatten()
    X_train[i] = train_img_f
    y_train[i] = train_data.loc[i, "DC"]   

    # 左右反転させたものを訓練データに追加
    train_img_lr = np.fliplr(train_img)
    train_img_lr_f = train_img_lr.flatten()
    X_train[i + train_len] = train_img_lr_f
    y_train[i + train_len] = train_data.loc[i, "DC"]

    # 上下反転させたものを訓練データに追加
    train_img_ud = np.flipud(train_img) 
    train_img_ud_f = train_img_ud.flatten()
    X_train[i + train_len * 2] = train_img_ud_f
    y_train[i + train_len * 2] = train_data.loc[i, "DC"]

    # 180度回転させたものを訓練データに追加
    train_img_180 = np.rot90(train_img, 2)
    train_img_180_f = train_img_180.flatten()
    X_train[i + train_len * 3] = train_img_180_f
    y_train[i + train_len * 3] = train_data.loc[i, "DC"]

- テストデータの作成

In [4]:
test_len = len(test_data)

X_test = np.empty((test_len, 75 * 75), dtype=np.uint8)
y_test = np.empty(test_len, dtype=np.uint8)

for i in range(test_len):
    name = test_data.loc[i, "File name"]
    test_img = Image.open(f"dc_photos/test/{name}.jpg").convert("L")
    test_img = np.array(test_img)
    test_img_f = test_img.flatten()
    X_test[i] = test_img_f
    y_test[i] = test_data.loc[i, "DC"]

## 3.モデルを作って学習する

In [5]:
classifier = SVC(kernel="linear")
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [6]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8)

In [7]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8)

## 4.判定

In [8]:
metrics.confusion_matrix(y_test, y_pred)

array([[36, 14],
       [ 5, 45]])

In [9]:
# 正答率
metrics.accuracy_score(y_test, y_pred)

0.81

In [10]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.72      0.79        50
           1       0.76      0.90      0.83        50

    accuracy                           0.81       100
   macro avg       0.82      0.81      0.81       100
weighted avg       0.82      0.81      0.81       100

